In [44]:
import pandas as pd
from tqdm import trange
from tqdm import tqdm
import tweepy
import pickle
import networkx as nx 

In [4]:
congress = pd.read_pickle('../Data/Interim/congress.pkl')

In [5]:
congress.head()

,user_id,user_name,id,created_at,text,retweet,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id
787068,516880804,Rep. Betty McCollum,824406510558027776,2017-01-26 00:00:03,I will strongly oppose spending a single cent ...,False,47,196,NaN,NaN
787069,22523087,Jim Cooper,824407835756916737,2017-01-26 00:05:19,Cooper Statement on Trump Immigration Actions ...,False,51,140,NaN,NaN
787070,815733290955112448,Rep. Pramila Jayapal,824408508510531584,2017-01-26 00:07:59,Today and every day I am proud of the @CityofS...,False,15,50,NaN,NaN
787071,161791703,Rep. Bobby Scott,824408755756560384,2017-01-26 00:08:58,Great to see Dr. Bruce Rubin of @ODUnow &amp; ...,False,1,6,NaN,NaN
787072,815733290955112448,Rep. Pramila Jayapal,824408887059099648,2017-01-26 00:09:30,"""We reject the false choice between safety and...",MayorEdMurray,54,0,NaN,NaN


In [6]:
trump = pd.read_pickle('../Data/Interim/trump.pkl')

In [7]:
trump.head()

,user_id,user_name,id,created_at,text,retweet,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id
0,25073877,Donald J. Trump,1079497288605683712,2018-12-30 21:59:44,President and Mrs. Obama built/has a ten foot ...,False,51992,209013,NaN,NaN
1,25073877,Donald J. Trump,1078396200842395648,2018-12-27 21:04:24,I totally agree! https://t.co/KO8E3bfWfn,False,36687,155505,NaN,NaN
2,25073877,Donald J. Trump,1079096156708319232,2018-12-29 19:25:47,For those that naively ask why didn’t the Repu...,False,28424,126667,NaN,NaN
3,25073877,Donald J. Trump,1076628127525294080,2018-12-22 23:58:43,I will not be going to Florida because of the ...,False,19111,102517,NaN,NaN
4,25073877,Donald J. Trump,1079398853684707328,2018-12-30 15:28:35,Veterans on President Trump’s handling of Bord...,False,21387,109701,NaN,NaN


In [8]:
# Concatinating the congress with Trump
congress_tweets = pd.concat([congress, trump])

In [9]:
congress_tweets.shape

(2850576, 10)

In [10]:
# Removing duplicates
congress_tweets.drop_duplicates(keep='first', inplace=True)

In [11]:
congress_tweets.shape

(2192037, 10)

In [12]:
# Getting handles
medias = pd.read_table('../Data/Raw/LargestMedia.csv', sep=';')
twitter_handles = pd.read_table('../Data/Processed/Twitter_Handles_updated.csv', sep = ',')

In [13]:
medias['State'] = None
medias['Party'] = None
medias['Type'] = 'Media'
medias.rename(columns={'Twitter name': 'Twitter', 'Media': 'Name'}, inplace=True)

In [14]:
Data_Full = pd.concat([twitter_handles, medias])

In [59]:
sorted(Data_Full.Twitter)

['ABC',
 'AlLawsonJr',
 'AndyBiggs4AZ',
 'AustinScottGA08',
 'BennieGThompson',
 'BettyMcCollum04',
 'BillCassidy',
 'BillPascrell',
 'BobbyScott',
 'BradSherman',
 'BreitbartNews',
 'BrianMastFL',
 'BuddforCongress',
 'CBSNews',
 'CNN',
 'Call_Me_Dutch',
 'CaptClayHIggins',
 'CarbajalSalud',
 'CatherineForNV',
 'CharlieCrist',
 'ChrisCoons',
 'ChrisMurphyCT',
 'ChrisVanHollen',
 'ChuckGrassley',
 'ClairecMc',
 'ClaudiaTenney',
 'ColleenHanabusa',
 'ConawayTX11',
 'CongBoyle',
 'CongMikeSimpson',
 'CongPalazzo',
 'CongressmanGT',
 'CongressmanHice',
 'CoryBooker',
 'DanaRohrabacher',
 'DarrellIssa',
 'DavidKustoff',
 'DesJarlaisTN04',
 'DevinNunes',
 'DonNorcross4NJ',
 'Donald_McEachin',
 'DorisMatsui',
 'DougJones',
 'DrNealDunnFL2',
 'DrPhilRoe',
 'DrewFergusonGA',
 'EleanorNorton',
 'ElectLaHood',
 'EspaillatNY',
 'FoxNews',
 'FrancisRooney',
 'FrankPallone',
 'GKButterfield',
 'GOPLeader',
 'GReschenthaler',
 'GarrettforVA',
 'GeorgeHolding',
 'GerryConnolly',
 'GregForMT',
 'Gregg

In [16]:
congress_tweets[congress_tweets['retweet'] != False].shape

(529703, 10)

In [17]:
retweets_pd = congress_tweets[congress_tweets['retweet'] != False]

In [18]:
#retweets_pd['retweet'].iloc[1] in Data_Full['Twitter']

Data_Full[Data_Full['Twitter'] == 'KamalaHarris']
#[i for i in retweets_pd['retweet'] if i.isin(Data_Full['Twitter'])]


,Twitter,State,Party,Type,Name
117,KamalaHarris,CA,Democrat,Senator,Kamala Harris


In [20]:
retweets_pd = retweets_pd[retweets_pd['retweet'].isin(Data_Full.Twitter)]

In [21]:
retweets_pd.shape

(95139, 10)

In [22]:
# Who retweets who
retweets_pd.head()

,user_id,user_name,id,created_at,text,retweet,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id
787078,29501253,Adam Schiff,824411350420127745,2017-01-26 00:19:17,Executive orders to break up families in Calif...,KamalaHarris,757,0,NaN,NaN
787222,41417564,Rep. Lloyd Smucker,824583733693075457,2017-01-26 11:44:16,We will use the Congressional Review Act to re...,GOPLeader,230,0,NaN,NaN
787232,2863006655,Brenda Lawrence,824598117857292290,2017-01-26 12:41:26,A day that I will always remember. #tbt #resis...,RepBonnie,12,0,NaN,NaN
787242,231510077,Joaquin Castro,824608733456138243,2017-01-26 13:23:37,Chaffetz: I’m looking into Trump D.C. hotel le...,politico,117,0,NaN,NaN
787249,3353670647,Archive: Dan Donovan,824616666818629632,2017-01-26 13:55:08,The #GOPretreat is about bringing the @HouseGO...,cathymcmorris,21,0,NaN,NaN


In [53]:
df_retweets = pd.DataFrame(data = 0, index = Data_Full['Twitter'], columns = Data_Full['Twitter'])

In [56]:
twitter_name_dict = {name: twitter_handle for twitter_handle, name in zip(Data_Full['Twitter'], Data_Full['Name'])}

In [58]:
for idx, retweet in tqdm(retweets_pd.iterrows(), total=len(retweets_pd)):
    row = twitter_name_dict[retweet['user_name']]
    col = retweet['retweet']

    df_retweets.loc[row, col] += 1



  0%|          | 1/95139 [00:01<42:38:20,  1.61s/it]


KeyError: 'Rep. Lloyd Smucker'

In [48]:
df_retweets

Name,Aumua Radewagen,Jeff Flake,Jon Kyl,Peter Aguilar,Tony Cardenas,Darrell Issa,Steve Knight,Dana Rohrabacher,Ed Royce,David Valadao,...,Bloomberg,The New York Times,USA Today Politics,Breitbart News,TIME,The Hill,POLITICO,The Wall Street Journal,The Economist,HuffPost
Name,,,,,,,,,,,,,,,,,,,,,
Aumua Radewagen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jeff Flake,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jon Kyl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peter Aguilar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tony Cardenas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Hill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
POLITICO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Wall Street Journal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
